In [1]:
import platform
platform.python_version()

'3.7.2'

In [2]:
import numpy as np
print("NumPy Version:",np.__version__)

NumPy Version: 1.15.4


In [3]:
from tabulate import tabulate

# Use Case Vectorization:

Consider, the expression
\begin{equation}
S =\sum\limits_{k=0}^{100}{\sqrt{\frac{k\pi }{100}}\sin }\left( \frac{k\pi }{100} \right)
\end{equation}

The direct approach is to evaluate the sum in a loop, resulting in the following *scalar* code:

In [4]:
from math import sqrt, sin, pi
x, S = 0.0, 0.0
for k in range(101):
    S += sqrt(x) * sin(x)
    x += 0.01*pi
print(S)

77.51389798916522


In [5]:
# Using list comprehension
x1 = [k*0.01*pi for k in range(101)]
S1 = [sqrt(y)*sin(y) for y in x1]
print(sum(S1))

77.5138979891651


In [6]:
# The vectorized version of the algorithm is
x = np.linspace(0, np.pi, 101)
# x = np.r_[0:np.pi+0.01:0.01*np.pi]
S = np.add.reduce(np.sqrt(x)*np.sin(x))
print(S)

77.51389798916512


# Use Case np.where():

$
    U\left( x \right)=\left\{ \begin{align}
      & \frac{b-x}{b-a},\text{ if }a\le x\le b \\ 
     & 0,\text{       otherwise} \\ 
    \end{align} \right.
$

In usual way, the code of the function is:

```python
def U(x, a, b):
    for val in x:
        if a <= val <= b:
            return (b-x)/(b-a)
        else:
            return 0
```     

In [7]:
def U(x,a,b):
    return np.where(np.logical_and(x >= a, x <= b), (b-x)/(b-a), 0)

In [8]:
def U1(x, a, b):    
    return np.where((x >= a) & (x <= b), (b-x)/(b-a), 0)

In [9]:
np.random.seed(42)
y = np.random.normal(5.5, 3, size=5)
y

array([ 6.99014246,  5.0852071 ,  7.44306561, 10.06908957,  4.79753988])

In [10]:
headers = ["y", "U(y)", "U1(y)"]
table = tabulate(np.c_[y, U(y, 4.5, 6.5), U1(y, 4.5, 6.5)], 
    headers, tablefmt="fancy_grid")
print(table)

╒══════════╤══════════╤══════════╕
│        y │     U(y) │    U1(y) │
╞══════════╪══════════╪══════════╡
│  6.99014 │ 0        │ 0        │
├──────────┼──────────┼──────────┤
│  5.08521 │ 0.707396 │ 0.707396 │
├──────────┼──────────┼──────────┤
│  7.44307 │ 0        │ 0        │
├──────────┼──────────┼──────────┤
│ 10.0691  │ 0        │ 0        │
├──────────┼──────────┼──────────┤
│  4.79754 │ 0.85123  │ 0.85123  │
╘══════════╧══════════╧══════════╛


# Performing Coin flips simulation

- Assign 0 as tails and 1 as heads. So flippling one coin is given as:

In [11]:
rand = np.random.RandomState(42)
rand.randint(low=0, high=2, size=1)

array([0])

In [12]:
# throwing a coin 10 times: 0 is tail, 1 heads
experiment = rand.randint(0,2, size=10)
print(experiment)
print(experiment.sum())

[1 0 0 0 1 0 0 0 1 0]
3


Let's continue this experiment, many times, say 1,000 times, to find out the distribution of the number of heads when throwing 10 coins at a time.  

In fact, it follows Binomial distribution with mean $np$ and variance $np(1-p)$. 
In this case, $n = 10$ and $p = 0.5$

**Binomial Distribution:**
\begin{equation}
P\left( X=r \right){{=}_{n}}{{c}_{r}}{{p}^{r}}{{\left( 1-p \right)}^{n-r}}
\end{equation}

**Expected Frequency:**

\begin{equation}
 N \times P\left( X=r \right)
\end{equation}

In [13]:
#Each column of this matrix will be one 10-tosses simulation
coin_matrix = rand.randint(0,2,size=(1000,10)) 
coin_matrix[:5,:]

array([[0, 0, 0, 1, 0, 1, 1, 1, 0, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 1, 1, 0],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 0]])

Here, the first five rows are the first five results of the 10 coins that we flip out of the 1,000 results that are in the actual matrix with 1,000 rows.

To calculate how many heads we got in every experiment, we have to sum all the rows.

In [14]:
counts = coin_matrix.sum(axis=1)
print(counts.shape)
print(counts[:20])  #  first 20 elements in the array, which contain the number of head

(1000,)
[5 8 4 7 4 5 6 6 8 9 6 6 1 4 3 6 3 3 3 5]


In [15]:
from math import factorial as f
def nCr(n, r):    
    return f(n) // f(r) // f(n-r)

In [16]:
# frequency distribution
no_of_heads = np.arange(11)
observed_freq = np.bincount(counts)
n, p, N = 10, 0.5, 1000
expected_freq = [N*nCr(n,r)*p**n for r in range(11)]
headers = ["No. of heads", "Observed freq", "Expected freq"]
table = tabulate(np.c_[no_of_heads, observed_freq, np.around(expected_freq)], 
    headers, tablefmt="fancy_grid")
print(table)
print("sum of Observed freq:", np.sum(observed_freq))
print("sum of Expected freq:", np.sum(expected_freq))

╒════════════════╤═════════════════╤═════════════════╕
│   No. of heads │   Observed freq │   Expected freq │
╞════════════════╪═════════════════╪═════════════════╡
│              0 │               0 │               1 │
├────────────────┼─────────────────┼─────────────────┤
│              1 │               8 │              10 │
├────────────────┼─────────────────┼─────────────────┤
│              2 │              39 │              44 │
├────────────────┼─────────────────┼─────────────────┤
│              3 │             134 │             117 │
├────────────────┼─────────────────┼─────────────────┤
│              4 │             210 │             205 │
├────────────────┼─────────────────┼─────────────────┤
│              5 │             237 │             246 │
├────────────────┼─────────────────┼─────────────────┤
│              6 │             192 │             205 │
├────────────────┼─────────────────┼─────────────────┤
│              7 │             129 │             117 │
├─────────

In [17]:
ex_stat = [counts.mean(), counts.var(), counts.std()]
BD_stat = [n*p, n*p*(1-p), np.sqrt(n*p*(1-p))]
row_head = ["Mean", "Variance", "Standard Deviation"]
headers = [" ", "Binomial Dist", "Experiemnt"]
table = tabulate(np.c_[row_head, BD_stat, ex_stat], 
    headers, tablefmt="fancy_grid")
print(table)

╒════════════════════╤═════════════════╤══════════════╕
│                    │   Binomial Dist │   Experiemnt │
╞════════════════════╪═════════════════╪══════════════╡
│ Mean               │         5       │      4.989   │
├────────────────────┼─────────────────┼──────────────┤
│ Variance           │         2.5     │      2.48488 │
├────────────────────┼─────────────────┼──────────────┤
│ Standard Deviation │         1.58114 │      1.57635 │
╘════════════════════╧═════════════════╧══════════════╛


**Note:** NumPy has a reasonable linear algebra package that performs standard linear algebra functions. Interested reader can explore `np.linalg`. 